# Query graph en plot polygonen
- We gebruiken de folium en geopandas libraries hiervoor
- Begin met een sparql query die polygonen oplevert (multipolygonen moet ook kunnen maar nog niet geprobeerd)

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

KKG_API = "https://api.labs.kadaster.nl/datasets/dst/kkg/services/default-old/sparql"
BGT_API = "https://api.labs.kadaster.nl/datasets/bgt/lv/services/default/sparql"
BAG_API = "https://api.labs.kadaster.nl/datasets/bag/lv/services/default/sparql"

def result_tabel(result):
    """"Table output, kan altijd
        Indien een variabele leeg is geef dan None terug.
    """
    import geopandas as pd
    try:
        df = pd.GeoDataFrame(data=[ [ r[v]['value'] if v in r.keys() else None for v in result['head']['vars'] ]  for r in result['results']['bindings'] ], columns=result['head']['vars'])
        return df
    except Exception as e:
        return str(e)
    return df


# Sparql query
definieer hier de sparql query die op het sparql eind point draait.

In [2]:
query = """
# Laat mij alle monumenten in mijn gemeente zien

PREFIX kad-con: <https://data.kkg.kadaster.nl/kad/model/con/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
prefix bag: <http://bag.basisregistraties.overheid.nl/def/bag#>
prefix brt: <http://brt.basisregistraties.overheid.nl/def/top10nl#>
prefix geo: <http://www.opengis.net/ont/geosparql#>
prefix kad: <https://data.kkg.kadaster.nl/kad/model/def/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix sor: <https://data.kkg.kadaster.nl/sor/model/def/>
prefix wbk: <https://data.labs.kadaster.nl/cbs/wbk/vocab/>

SELECT distinct  ?suffix_uri ?geometrie_wgs84 ?gemeenteNaam
WHERE
{
	#BIND ('Goirle'@nl AS ?gemeenteNaam)
	BIND (%s AS ?gemeenteNaam)

  	# Beperkingen tbv monumenten
  	VALUES ?grondslag {
    	kad-con:GG 
    	kad-con:GWA 
  	}
  
  	?suffix_uri 
    	a sor:Gebouw ;
    	^kad:gevestigdOp ?beperking ;
  		geo:hasGeometry ?polygoon ;
        sor:bronhouder ?bronhouder .

  	?bronhouder skos:prefLabel ?gemeenteNaam .

   	?beperking 
  		kad:grondslag ?grondslag .
  
    ?polygoon geo:asWKT ?geometrie_wgs84;
     	rdfs:isDefinedBy bag: .
}

""" % "'Goirle'@nl"

In [3]:
sparql = SPARQLWrapper(KKG_API)
sparql.setReturnFormat(JSON)
sparql.setQuery(query)
result = sparql.query()
result

In [4]:
sparql = SPARQLWrapper(KKG_API)
sparql.setReturnFormat(JSON)
sparql.setQuery(query)
result = sparql.queryAndConvert()

df = result_tabel(result)

In [5]:
import geopandas as gpd
df.set_geometry(gpd.GeoSeries.from_wkt(df['geometrie_wgs84']), inplace=True)
df.head()

,suffix_uri,geometrie_wgs84,gemeenteNaam,geometry
0,https://data.kkg.kadaster.nl/id/gebouw/0785100...,"POLYGON((5.0073741582861 51.515920063651,5.007...",Goirle,"POLYGON ((5.00737 51.51592, 5.00743 51.51608, ..."
1,https://data.kkg.kadaster.nl/id/gebouw/0785100...,"POLYGON((5.0671237470116 51.519432957527,5.067...",Goirle,"POLYGON ((5.06712 51.51943, 5.06722 51.51947, ..."
2,https://data.kkg.kadaster.nl/id/gebouw/0785100...,"POLYGON((5.0554084836559 51.507966608095,5.055...",Goirle,"POLYGON ((5.05541 51.50797, 5.05544 51.50801, ..."
3,https://data.kkg.kadaster.nl/id/gebouw/0785100...,"POLYGON((5.0696380453264 51.518930365297,5.069...",Goirle,"POLYGON ((5.06964 51.51893, 5.06960 51.51891, ..."
4,https://data.kkg.kadaster.nl/id/gebouw/0785100...,"POLYGON((5.0152308795309 51.532322381604,5.015...",Goirle,"POLYGON ((5.01523 51.53232, 5.01529 51.53239, ..."


In [6]:
import folium

m = folium.Map(location=[51.52,5.06], zoom_start=12)

for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = folium.GeoJson(
        data=sim_geo.to_json(), 
    )
    folium.Popup(r["suffix_uri"]).add_to(geo_j)
    geo_j.add_to(m)
m
